# Responses APIでのファイル検索ツールの使用

RAGは複雑になりがちですが、PDFファイル間での検索は複雑である必要はありません。現在最も採用されている選択肢の一つは、PDFを解析し、チャンク化戦略を定義し、それらのチャンクをストレージプロバイダーにアップロードし、それらのテキストチャンクに対して埋め込みを実行し、それらの埋め込みをベクターデータベースに保存することです。そして、それはセットアップのみです — LLMワークフローでコンテンツを取得することも複数のステップが必要です。

ここで、Responses APIで使用できるホスト型ツールであるファイル検索が登場します。これにより、ナレッジベースを検索し、取得したコンテンツに基づいて回答を生成することができます。このクックブックでは、これらのPDFをOpenAIのベクターストアにアップロードし、ファイル検索を使用してこのベクターストアから追加のコンテキストを取得し、最初のステップで生成した質問に答えます。その後、OpenAIのブログ（[openai.com/news](https://openai.com/news)）から抽出したPDFに基づいて、最初に小さな質問セットを作成します。

_ファイル検索は以前Assistants APIで利用可能でした。現在は新しいResponses APIで利用可能になっており、このAPIはステートフルまたはステートレスにでき、メタデータフィルタリングなどの新機能も備えています_

# PDFを使用したベクターストアの作成

In [ ]:
!pip install PyPDF2 pandas tqdm openai -q

In [1]:
from openai import OpenAI
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm
import concurrent
import PyPDF2
import os
import pandas as pd
import base64

client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))
dir_pdfs = 'openai_blog_pdfs' # have those PDFs stored locally here
pdf_files = [os.path.join(dir_pdfs, f) for f in os.listdir(dir_pdfs)]

OpenAI APIでVector Storeを作成し、PDFをVector Storeにアップロードします。OpenAIはそれらのPDFを読み取り、コンテンツを複数のテキストチャンクに分割し、それらに対してembeddingを実行し、そのembeddingとテキストをVector Storeに保存します。これにより、クエリに基づいて関連するコンテンツを返すためにこのVector Storeにクエリを実行できるようになります。

In [2]:
def upload_single_pdf(file_path: str, vector_store_id: str):
    file_name = os.path.basename(file_path)
    try:
        file_response = client.files.create(file=open(file_path, 'rb'), purpose="assistants")
        attach_response = client.vector_stores.files.create(
            vector_store_id=vector_store_id,
            file_id=file_response.id
        )
        return {"file": file_name, "status": "success"}
    except Exception as e:
        print(f"Error with {file_name}: {str(e)}")
        return {"file": file_name, "status": "failed", "error": str(e)}

def upload_pdf_files_to_vector_store(vector_store_id: str):
    pdf_files = [os.path.join(dir_pdfs, f) for f in os.listdir(dir_pdfs)]
    stats = {"total_files": len(pdf_files), "successful_uploads": 0, "failed_uploads": 0, "errors": []}
    
    print(f"{len(pdf_files)} PDF files to process. Uploading in parallel...")

    with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
        futures = {executor.submit(upload_single_pdf, file_path, vector_store_id): file_path for file_path in pdf_files}
        for future in tqdm(concurrent.futures.as_completed(futures), total=len(pdf_files)):
            result = future.result()
            if result["status"] == "success":
                stats["successful_uploads"] += 1
            else:
                stats["failed_uploads"] += 1
                stats["errors"].append(result)

    return stats

def create_vector_store(store_name: str) -> dict:
    try:
        vector_store = client.vector_stores.create(name=store_name)
        details = {
            "id": vector_store.id,
            "name": vector_store.name,
            "created_at": vector_store.created_at,
            "file_count": vector_store.file_counts.completed
        }
        print("Vector store created:", details)
        return details
    except Exception as e:
        print(f"Error creating vector store: {e}")
        return {}

In [3]:
store_name = "openai_blog_store"
vector_store_details = create_vector_store(store_name)
upload_pdf_files_to_vector_store(vector_store_details["id"])

Vector store created: {'id': 'vs_67d06b9b9a9c8191bafd456cf2364ce3', 'name': 'openai_blog_store', 'created_at': 1741712283, 'file_count': 0}
21 PDF files to process. Uploading in parallel...


100%|███████████████████████████████| 21/21 [00:09<00:00,  2.32it/s]


{'total_files': 21,
 'successful_uploads': 21,
 'failed_uploads': 0,
 'errors': []}

# スタンドアロンベクター検索

ベクターストアの準備が完了したので、ベクターストアに直接クエリを実行し、特定のクエリに関連するコンテンツを取得できるようになりました。新しい[vector search API](https://platform.openai.com/docs/api-reference/vector-stores/search)を使用することで、LLMクエリに必ずしも統合することなく、ナレッジベースから関連するアイテムを見つけることができます。

In [4]:
query = "What's Deep Research?"
search_results = client.vector_stores.search(
    vector_store_id=vector_store_details['id'],
    query=query
)

In [5]:
for result in search_results.data:
    print(str(len(result.content[0].text)) + ' of character of content from ' + result.filename + ' with a relevant score of ' + str(result.score))

3502 of character of content from Introducing deep research _ OpenAI.pdf with a relevant score of 0.9813588865322393
3493 of character of content from Introducing deep research _ OpenAI.pdf with a relevant score of 0.9522476825143714
3634 of character of content from Introducing deep research _ OpenAI.pdf with a relevant score of 0.9397930296526796
2774 of character of content from Introducing deep research _ OpenAI.pdf with a relevant score of 0.9101975747303771
3474 of character of content from Deep research System Card _ OpenAI.pdf with a relevant score of 0.9036647613464299
3123 of character of content from Introducing deep research _ OpenAI.pdf with a relevant score of 0.887120981288272
3343 of character of content from Introducing deep research _ OpenAI.pdf with a relevant score of 0.8448454849432881
3262 of character of content from Introducing deep research _ OpenAI.pdf with a relevant score of 0.791345286655509
3271 of character of content from Introducing deep research _ Open

検索クエリから異なるサイズ（そして内部的には異なるテキスト）が返されていることがわかります。これらはすべて、ハイブリッド検索を使用するランカーによって計算された異なる関連性スコアを持っています。

# 単一のAPI呼び出しでLLMと検索結果を統合する

しかし、ベクトルストアにクエリを実行してからそのデータをResponsesまたはChat Completion API呼び出しに渡すのではなく、この検索結果をLLMクエリで使用するより便利な方法は、OpenAI Responses APIの一部として`file_search`ツールを使用することです。

In [7]:
query = "What's Deep Research?"
response = client.responses.create(
    input= query,
    model="gpt-4o-mini",
    tools=[{
        "type": "file_search",
        "vector_store_ids": [vector_store_details['id']],
    }]
)

# Extract annotations from the response
annotations = response.output[1].content[0].annotations
    
# Get top-k retrieved filenames
retrieved_files = set([result.filename for result in annotations])

print(f'Files used: {retrieved_files}')
print('Response:')
print(response.output[1].content[0].text) # 0 being the filesearch call

Files used: {'Introducing deep research _ OpenAI.pdf'}
Response:
Deep Research is a new capability introduced by OpenAI that allows users to conduct complex, multi-step research tasks on the internet efficiently. Key features include:

1. **Autonomous Research**: Deep Research acts as an independent agent that synthesizes vast amounts of information across the web, enabling users to receive comprehensive reports similar to those produced by a research analyst.

2. **Multi-Step Reasoning**: It performs deep analysis by finding, interpreting, and synthesizing data from various sources, including text, images, and PDFs.

3. **Application Areas**: Especially useful for professionals in fields such as finance, science, policy, and engineering, as well as for consumers seeking detailed information for purchases.

4. **Efficiency**: The output is fully documented with citations, making it easy to verify information, and it significantly speeds up research processes that would otherwise take h

`gpt-4o-mini`がOpenAIのDeep Researchに関するより最新で専門的な知識を必要とするクエリに回答できることがわかります。最も関連性の高いテキストのチャンクを含む`Introducing deep research _ OpenAI.pdf`ファイルのコンテンツを使用しました。取得されたテキストのチャンクをさらに深く分析したい場合は、クエリに`include=["output[*].file_search_call.search_results"]`を追加することで、検索エンジンによって返された異なるテキストを分析することもできます。

# パフォーマンスの評価

これらの情報検索システムにとって重要なのは、回答のために取得されたファイルの関連性と品質を測定することです。このクックブックの以下のステップでは、評価データセットを生成し、この生成されたデータセットに対してさまざまなメトリクスを計算することで構成されます。これは不完全なアプローチであり、独自のユースケースには人間が検証した評価データセットを用意することを常に推奨しますが、これらを評価する方法論を示します。生成される質問の一部が汎用的（例：この文書で主要な関係者が述べていることは何か）である可能性があり、私たちの検索テストではその質問がどの文書のために生成されたかを把握するのが困難になるため、不完全になります。

## 評価データの生成

ローカルに保存されているPDFを読み取り、その文書でのみ回答可能な質問を生成する関数を作成します。これにより、後で使用できる評価データセットを作成します。

In [8]:
def extract_text_from_pdf(pdf_path):
    text = ""
    try:
        with open(pdf_path, "rb") as f:
            reader = PyPDF2.PdfReader(f)
            for page in reader.pages:
                page_text = page.extract_text()
                if page_text:
                    text += page_text
    except Exception as e:
        print(f"Error reading {pdf_path}: {e}")
    return text

def generate_questions(pdf_path):
    text = extract_text_from_pdf(pdf_path)

    prompt = (
        "Can you generate a question that can only be answered from this document?:\n"
        f"{text}\n\n"
    )

    response = client.responses.create(
        input=prompt,
        model="gpt-4o",
    )

    question = response.output[0].content[0].text

    return question

最初のPDFファイルに対して関数`generate_question`を実行すると、どのような質問が生成されるかを確認することができます。

In [9]:
generate_questions(pdf_files[0])

'What new capabilities will ChatGPT have as a result of the partnership between OpenAI and Schibsted Media Group?'

これで、ローカルに保存されているすべてのPDFに対して、すべての質問を生成できるようになりました。

In [10]:
# Generate questions for each PDF and store in a dictionary
questions_dict = {}
for pdf_path in pdf_files:
    questions = generate_questions(pdf_path)
    questions_dict[os.path.basename(pdf_path)] = questions

In [11]:
questions_dict

{'OpenAI partners with Schibsted Media Group _ OpenAI.pdf': 'What is the purpose of the partnership between Schibsted Media Group and OpenAI announced on February 10, 2025?',
 'OpenAI and the CSU system bring AI to 500,000 students & faculty _ OpenAI.pdf': 'What significant milestone did the California State University system achieve by partnering with OpenAI, making it the first of its kind in the United States?',
 '1,000 Scientist AI Jam Session _ OpenAI.pdf': 'What was the specific AI model used during the "1,000 Scientist AI Jam Session" event across the nine national labs?',
 'Announcing The Stargate Project _ OpenAI.pdf': 'What are the initial equity funders and lead partners in The Stargate Project announced by OpenAI, and who holds the financial and operational responsibilities?',
 'Introducing Operator _ OpenAI.pdf': 'What is the name of the new model that powers the Operator agent introduced by OpenAI?',
 'Introducing NextGenAI _ OpenAI.pdf': 'What major initiative did OpenAI

現在、`filename:question`の辞書があり、これをループして文書を提供することなくgpt-4o(-mini)に質問できます。gpt-4oはVector Store内の関連する文書を見つけることができるはずです。

## 評価

辞書をデータフレームに変換し、gpt-4o-miniを使用して処理します。期待されるファイルを確認します。

In [12]:
rows = []
for filename, query in questions_dict.items():
    rows.append({"query": query, "_id": filename.replace(".pdf", "")})

# Metrics evaluation parameters
k = 5
total_queries = len(rows)
correct_retrievals_at_k = 0
reciprocal_ranks = []
average_precisions = []

def process_query(row):
    query = row['query']
    expected_filename = row['_id'] + '.pdf'
    # Call file_search via Responses API
    response = client.responses.create(
        input=query,
        model="gpt-4o-mini",
        tools=[{
            "type": "file_search",
            "vector_store_ids": [vector_store_details['id']],
            "max_num_results": k,
        }],
        tool_choice="required" # it will force the file_search, while not necessary, it's better to enforce it as this is what we're testing
    )
    # Extract annotations from the response
    annotations = None
    if hasattr(response.output[1], 'content') and response.output[1].content:
        annotations = response.output[1].content[0].annotations
    elif hasattr(response.output[1], 'annotations'):
        annotations = response.output[1].annotations

    if annotations is None:
        print(f"No annotations for query: {query}")
        return False, 0, 0

    # Get top-k retrieved filenames
    retrieved_files = [result.filename for result in annotations[:k]]
    if expected_filename in retrieved_files:
        rank = retrieved_files.index(expected_filename) + 1
        rr = 1 / rank
        correct = True
    else:
        rr = 0
        correct = False

    # Calculate Average Precision
    precisions = []
    num_relevant = 0
    for i, fname in enumerate(retrieved_files):
        if fname == expected_filename:
            num_relevant += 1
            precisions.append(num_relevant / (i + 1))
    avg_precision = sum(precisions) / len(precisions) if precisions else 0
    
    if expected_filename not in retrieved_files:
        print("Expected file NOT found in the retrieved files!")
        
    if retrieved_files and retrieved_files[0] != expected_filename:
        print(f"Query: {query}")
        print(f"Expected file: {expected_filename}")
        print(f"First retrieved file: {retrieved_files[0]}")
        print(f"Retrieved files: {retrieved_files}")
        print("-" * 50)
    
    
    return correct, rr, avg_precision

In [13]:
process_query(rows[0])

(True, 1.0, 1.0)

この例では、Recall & Precisionが1となっており、私たちのファイルが1位にランクされているため、この例ではMRRとMAP = 1となっています。

これで、質問セットに対してこの処理を実行できます。

In [14]:
with ThreadPoolExecutor() as executor:
    results = list(tqdm(executor.map(process_query, rows), total=total_queries))

correct_retrievals_at_k = 0
reciprocal_ranks = []
average_precisions = []

for correct, rr, avg_precision in results:
    if correct:
        correct_retrievals_at_k += 1
    reciprocal_ranks.append(rr)
    average_precisions.append(avg_precision)

recall_at_k = correct_retrievals_at_k / total_queries
precision_at_k = recall_at_k  # In this context, same as recall
mrr = sum(reciprocal_ranks) / total_queries
map_score = sum(average_precisions) / total_queries

 62%|███████████████████▏           | 13/21 [00:07<00:03,  2.57it/s]

Expected file NOT found in the retrieved files!
Query: What is OpenAI's mission as stated in the document?
Expected file: Disrupting malicious uses of AI _ OpenAI.pdf
First retrieved file: Introducing the Intelligence Age _ OpenAI.pdf
Retrieved files: ['Introducing the Intelligence Age _ OpenAI.pdf']
--------------------------------------------------


 71%|██████████████████████▏        | 15/21 [00:14<00:06,  1.04s/it]

Expected file NOT found in the retrieved files!
Query: What is the purpose of the "Model Spec" document published by OpenAI for ChatGPT?
Expected file: The power of personalized AI _ OpenAI.pdf
First retrieved file: Sharing the latest Model Spec _ OpenAI.pdf
Retrieved files: ['Sharing the latest Model Spec _ OpenAI.pdf', 'Sharing the latest Model Spec _ OpenAI.pdf', 'Sharing the latest Model Spec _ OpenAI.pdf', 'Sharing the latest Model Spec _ OpenAI.pdf', 'Sharing the latest Model Spec _ OpenAI.pdf']
--------------------------------------------------


100%|███████████████████████████████| 21/21 [00:15<00:00,  1.38it/s]


上記でログ出力された結果は、評価データセットで1位にランクされることが期待されていたファイルが実際には1位にランクされなかった場合、または全く見つからなかった場合のいずれかを示しています。不完全な評価データセットから分かるように、一部の質問は汎用的で別の文書を期待していましたが、我々の検索システムはこの質問に対してその文書を具体的に取得しませんでした。

In [15]:
# Print the metrics with k
print(f"Metrics at k={k}:")
print(f"Recall@{k}: {recall_at_k:.4f}")
print(f"Precision@{k}: {precision_at_k:.4f}")
print(f"Mean Reciprocal Rank (MRR): {mrr:.4f}")
print(f"Mean Average Precision (MAP): {map_score:.4f}")

Metrics at k=5:
Recall@5: 0.9048
Precision@5: 0.9048
Mean Reciprocal Rank (MRR): 0.9048
Mean Average Precision (MAP): 0.8954


このクックブックでは、以下の方法を確認することができました：
- PDFコンテキスト詰め込み（4oのビジョンモダリティを活用）と従来のPDFリーダーを使用した評価データセットの生成
- ベクターストアの作成とPDFでの入力
- OpenAIのResponse APIの`file_search`ツール呼び出しで利用可能な、すぐに使えるRAGシステムを活用したクエリに対するLLM回答の取得
- Response APIの一部として、テキストのチャンクがどのように検索、ランク付け、使用されるかの理解
- 事前に生成された評価データセットでの精度、適合率、検索率、MRR、MAPの測定

Responsesでfile searchを使用することで、RAGアーキテクチャを簡素化し、新しいResponses APIを使用して単一のAPI呼び出しでこれを活用できます。ファイルストレージ、埋め込み、検索がすべて1つのツールに統合されています！